<a href="https://colab.research.google.com/github/JonNData/used-car-predictor/blob/master/notebooks/cl_price_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 

In [0]:
pip install category_encoders

     |████████████████████████████████| 81kB 2.6MB/s 


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import RobustScaler

from sklearn.linear_model import RidgeCV

from sklearn.ensemble import RandomForestRegressor

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Go to link, sign in, and paste in result

In [0]:
file_list = drive.ListFile({'q': "'1kE03N5LwmWDDsDescCPmlQxGsY7ZKEnT' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: craigslist_df_cleaned, id: 1NT6IPKm1EwNeH5PBVt8HqRMtO_oJemnC
title: Weekly_U.S._All_Grades_All_Formulations_Retail_Gasoline_Prices, id: 1cmGfdjaAd7RXYa_9S8RcraEDBA1fJKoKzB17KcaEo7Y
title: Weekly_U.S._All_Grades_All_Formulations_Retail_Gasoline_Prices.csv, id: 1pBHxMMMbUXZfFhNj5c2KqFq_dm4EkYOK
title: vehicles.csv, id: 12aa_5F_dECaFGjQR5KYZwToAqhHouUNi
title: carfullspecs.csv, id: 1Y9oQ0huvw3LYnbafa-AM7Ig5cOK95DkD
title: vehicles_cl.csv, id: 1BKZfd5WBLkryOSWI6t4g50bloTD8P6f3


In [0]:
downloaded = drive.CreateFile({'id':"1NT6IPKm1EwNeH5PBVt8HqRMtO_oJemnC"})  
downloaded.GetContentFile('craigslist_df_cleaned.csv')       

In [0]:
downloaded = drive.CreateFile({'id':"12aa_5F_dECaFGjQR5KYZwToAqhHouUNi"})  
downloaded.GetContentFile('vehicles.csv')       

In [0]:
df_orig = pd.read_csv('craigslist_df_cleaned.csv')
df_orig

,Unnamed: 0,price,year,manufacturer,model,condition,fuel,odometer,state
0,0,10299.0,2012.0,acura,tl,NaN,gas,90186.0,nc
1,2,9500.0,2011.0,jaguar,xf,excellent,gas,85000.0,nc
2,3,3995.0,2004.0,honda,element,NaN,gas,212526.0,nc
3,4,41988.0,2016.0,chevrolet,silverado k2500hd,NaN,gas,NaN,ne
4,5,12995.0,2015.0,kia,sportage,NaN,gas,85127.0,nc
...,...,...,...,...,...,...,...,...,...
458626,539753,12955.0,2015.0,mazda,cx-5,NaN,gas,74508.0,nc
458627,539755,13995.0,2016.0,nissan,frontier,NaN,gas,97545.0,nc
458628,539756,10995.0,2010.0,bmw,335i,NaN,gas,80328.0,nc
458629,539757,6495.0,2015.0,dodge,dart,NaN,other,83521.0,nc


In [0]:
df = df_orig.copy()
df.rename(columns={'Unnamed: 0': 'original_index'}, inplace=True)

In [0]:
df.describe()

,original_index,price,year,odometer
count,458631.000000,458631.000000,458631.000000,3.815300e+05
mean,269437.912121,13282.017940,2009.863936,1.026567e+05
std,155904.234715,9814.956853,8.601419,1.815791e+05
min,0.000000,199.000000,0.000000,0.000000e+00
25%,134560.000000,5590.000000,2007.000000,5.010400e+04
50%,268329.000000,10900.000000,2011.000000,9.608600e+04
75%,404575.500000,18500.000000,2015.000000,1.400000e+05
max,539758.000000,52000.000000,2021.000000,6.480922e+07


In [0]:
df.dtypes

original_index      int64
price             float64
year              float64
manufacturer       object
model              object
condition          object
fuel               object
odometer          float64
state              object
dtype: object

In [0]:
df.isnull().sum()

original_index         0
price                  0
year                   0
manufacturer           0
model                  0
condition         188861
fuel                2578
odometer           77101
state                  0
dtype: int64

## Split and define features + target 
3 features

In [0]:
train, test = train_test_split(df, test_size=0.15, random_state=11)

X_features = ['model', 'manufacturer', 'year']
target = 'price'

X_train = train[X_features]
y_train = train[target]

X_test = test[X_features]
y_test = test[target]

## Preprocessing

In [0]:
te = TargetEncoder()
X_train_encoded = te.fit_transform(X_train, y_train)
X_test_encoded = te.transform(X_test)

rs = RobustScaler()
X_train_scaled = rs.fit_transform(X_train_encoded)
X_test_scaled = rs.transform(X_test_encoded)

## Ridge Regression

In [0]:
rcv = RidgeCV()
rcv.fit(X_train_scaled, y_train)
rcv.score(X_train_scaled, y_train)

0.5476627579471244

In [0]:
y_pred_ridge = rcv.predict(X_test_scaled)
mean_absolute_error(y_test, y_pred_ridge)

4687.762296107629

## Random Forest

In [0]:
rf = RandomForestRegressor()
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

mean_absolute_error(y_test, y_pred_rf)

2658.140933144546

In [0]:
y_pred_rf = rf.predict(X_test_scaled)

mean_absolute_error(y_test, y_pred_rf)

2658.140933144546

## Fuzzy Wuzzy attemps

In [0]:
pip install fuzzywuzzy

In [0]:
from fuzzywuzzy import fuzz, process

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [0]:
df_epa = pd.read_csv('vehicles.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# Start with epa, compare models where year and make is the same

# for row in df_epa:
#   if df_epa.year == df_cl.year:
#     print(fuzz.ratio(df_epa['model'][row], df_cl['model'][row]))



In [0]:
df_cl = df

In [0]:
# Lets look at a simplified list

epa_models = sorted(df_epa['model'].unique())
cl_models = sorted(df_cl.model.unique())


for model in epa_models:
  for x  in range(len(cl_models)):
    print(fuzz.ratio(model, cl_models[x]))

In [0]:
epa_models[2888]

In [0]:
grand_cherokee_ratios = []
for model in cl_models:
  grand_cherokee_ratios.append(fuzz.ratio(epa_models[2888], model))
max(grand_cherokee_ratios)

In [0]:
grand_cherokee_ratios.index(84)

In [0]:
cl_models[18845]

'q50 red sport 400 awd'

In [0]:
df_epa['model'].nunique(), df['model'].nunique()

(4218, 26904)

In [0]:
# Input Make model Year get a prediction
# sample_model = 'sivics'

cl_models = sorted(df_cl.model.unique())

def match_models(sample_model):
  model_ratios = []
  for model in cl_models:
    model_ratios.append(fuzz.ratio(sample_model, model))
  max_match = max(model_ratios)
  index_of_match = model_ratios.index(max_match)
  return cl_models[index_of_match]

In [0]:
match_models('Wrangler Rubicon 4wd')

'wrangler rubicon 4wd'

In [0]:
def match_models(sample_model):
  model_ratios = []
  for model in cl_models:
    model_ratios.append(fuzz.ratio(sample_model, model))
  max_match = max(model_ratios)
  index_of_match = model_ratios.index(max_match)
  return cl_models[index_of_match]